In [21]:
import pandas as pd 

In [22]:
df = pd.read_csv(r"C:\Users\smitp\OneDrive\Desktop\spotify_millsongdata.csv")

In [23]:
df.head(5) 

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [24]:
df.isnull().sum() 

artist    0
song      0
link      0
text      0
dtype: int64

In [25]:
df = df.sample(10000).drop('link',axis=True).reset_index(drop=True)

Text Cleaning  / Text Preprocessing

In [26]:
df['text'] = df['text'].str.lower().replace(r'^\w\s',' ').replace(r'\n',' ',regex = True )

In [27]:
df.tail(5)

,artist,song,text
9995,Alice Cooper,Dead Babies,little betty ate a pound of aspirin \r she go...
9996,Alison Krauss,Gone Gone Gone (Done Moved On),some sunny day-hay baby \r when everything se...
9997,Bon Jovi,Edge Of A Broken Heart,"and there i stood, just like a soldier \r i w..."
9998,Barbra Streisand,He Touched Me,"he touched me, he put his hand near mine \r a..."
9999,Marillion,Exile On Princes Street,"due to the nature of this demo, no printed lyr..."


In [28]:
import nltk
from nltk.stem.porter import PorterStemmer

In [29]:
stemmer = PorterStemmer()

In [30]:
def token(txt):
    token = nltk.word_tokenize(txt)
    a = [stemmer.stem(w) for w in token]
    return " ".join(a)

In [31]:
token("you are beautiful,beauty")

'you are beauti , beauti'

In [32]:
df['text'].apply(lambda x:token(x))

0       we kept apart and connect by the same stretch ...
1       without your scheme my life it seem is empti ,...
2       turn anoth turn ... turn anoth turn , sp00l .....
3       onc i had a laugh and when i 'm older i will n...
4       i just want you around , i just want you aroun...
                              ...                        
9995    littl betti ate a pound of aspirin she got the...
9996    some sunni day-hay babi when everyth seem okay...
9997    and there i stood , just like a soldier i wa t...
9998    he touch me , he put hi hand near mine and the...
9999    due to the natur of thi demo , no print lyric ...
Name: text, Length: 10000, dtype: object

In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [34]:
tfid = TfidfVectorizer(analyzer='word',stop_words='english')

In [35]:
matrix = tfid.fit_transform(df['text'])

In [36]:
similar= cosine_similarity(matrix)

In [37]:
similar[0]

array([1.        , 0.        , 0.00653303, ..., 0.00616455, 0.00887381,
       0.00785788])

Recommender Function

In [38]:
def recommender(song_name):
    idx = df[df['song'] == song_name].index[0]
    distance = sorted(list(enumerate(similar[idx])), reverse=True, key=lambda x:x[1])
    song =[]
    for s_id in distance[1:11]:
        song.append(df.iloc[s_id[0]].song) 
    return song

In [39]:
import pickle

In [40]:
pickle.dump(similar,open("similarity","wb"))

In [41]:
pickle.dump(df,open("df","wb"))